Считайте датасет из файла train.csv (это данные о выживаемости на Титанике)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('drive/MyDrive/train.csv')

Выведите основную информацию о датасете: информацию о типах данных, число пропусков, средние значения и т.д. - 1 балл

In [ ]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Посчитайте процент выживаемости у каждого класса пассажиров (Pclass)

In [ ]:
survival = (df.groupby('Pclass')['Survived'].mean() #группируем по классу и количеству выживших, берём среднее
                                            .mul(100)#умножаем на 100
                                            .round()#округляем
                                            .reset_index(name='Процент выживаемости (%)'))#переименовываем столбец
print(survival)

   Pclass  Процент выживаемости (%)
0       1                      63.0
1       2                      47.0
2       3                      24.0


Выведите самое популярное мужское и самое популярное женское имя на корабле в каждом классе

In [ ]:
def extract_first_name(row):
    name = row['Name']
    if row['Sex'] == 'female':
          #женское имя записывается в формате [фамилия, титул. имя] ЛИБО [фамилия, титул и имя мужа (имя)]
          # если есть скобки то ищем имя в скобках
        if '(' in name and ')' in name:
            inside = name[name.find('(')+1 : name.find(')')].strip()
            return inside.split()[0]

    #извлекаем имя для всех остальных случаев. имя записывается в формате [фамилия, титул. имя]
    part = name.split('.', 1)[1].strip()#разделяем по точке чтобы извлечь имя, берём второй элемент (имя целиком), удаляем пробелы
    return part.split()[0]#делим ещё раз, на случай если имя состоит из двух частей, берём первое имя



df['first name'] = df.apply(extract_first_name, axis=1) #применяем функцию построчно
name_counts = (df.groupby(['Sex','Pclass', 'first name']) #группируем по классу имени и полу
                      .size()#считаем сколько имён
                     .reset_index(name='count'))#переименовываем столбец

top_names = (name_counts.sort_values(['Sex','Pclass','count'], ascending=[True, True, False]) #сортируем значения - класс по убыванию, счёт имени по возрастанию,
             #чтобы популярные были сверху
              .groupby(['Sex','Pclass']) #группирем по классу
              .head(1) # берём первый элемент
              .reset_index(drop = True))#сбрасываем индекс
print(top_names)



      Sex  Pclass first name  count
0  female       1  Elizabeth      5
1  female       2  Elizabeth      5
2  female       3       Anna      9
3    male       1    William     11
4    male       2    William      9
5    male       3    William     15


Выведите часть таблицы с пассажирами, возраст которых больше 44 лет - 1 балл

In [ ]:
age_44 = df[df['Age'] > 44]
print(age_44.to_string())

     PassengerId  Survived  Pclass                                                               Name     Sex   Age  SibSp  Parch        Ticket      Fare        Cabin Embarked    first name
6              7         0       1                                            McCarthy, Mr. Timothy J    male  54.0      0      0         17463   51.8625          E46        S       Timothy
11            12         1       1                                           Bonnell, Miss. Elizabeth  female  58.0      0      0        113783   26.5500         C103        S     Elizabeth
15            16         1       2                                   Hewlett, Mrs. (Mary D Kingcome)   female  55.0      0      0        248706   16.0000          NaN        S          Mary
33            34         0       2                                              Wheadon, Mr. Edward H    male  66.0      0      0    C.A. 24579   10.5000          NaN        S        Edward
52            53         1       1                

Выведите часть таблицы с пассажирами, возраст которых меньше 44 лет и которые мужского пола - 2 балла

In [ ]:
male_age_44 = df[(df['Sex']=='male') & (df['Age']>44)]
print(male_age_44.to_string())

     PassengerId  Survived  Pclass                                           Name   Sex   Age  SibSp  Parch       Ticket      Fare        Cabin Embarked    first name
6              7         0       1                        McCarthy, Mr. Timothy J  male  54.0      0      0        17463   51.8625          E46        S       Timothy
33            34         0       2                          Wheadon, Mr. Edward H  male  66.0      0      0   C.A. 24579   10.5000          NaN        S        Edward
54            55         0       1                 Ostby, Mr. Engelhart Cornelius  male  65.0      0      1       113509   61.9792          B30        C     Engelhart
62            63         0       1                    Harris, Mr. Henry Birkhardt  male  45.0      1      0        36973   83.4750          C83        S         Henry
92            93         0       1                    Chaffee, Mr. Herbert Fuller  male  46.0      1      0  W.E.P. 5734   61.1750          E31        S       Herber

Выведите количества n-местных кабин (в которых было 2, 3, 4, ... человека) - 1 бал

In [ ]:
cabin_counts = df['Cabin'].value_counts() #считаем количество каждых кабинок

print(f'Кабинок с 2 людьми: {(cabin_counts == 2).sum()} штук')#выводим сумму
print(f'Кабинок с 3 людьми: {(cabin_counts == 3).sum()} штук')
print(f'Кабинок с 4 людьми: {(cabin_counts >= 4).sum()} штук')

Кабинок с 2 людьми: 38 штук
Кабинок с 3 людьми: 5 штук
Кабинок с 4 людьми: 3 штук
